In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from decouple import config

In [4]:
from casestudy.update import auto

In [5]:
from casestudy.update.baseframe import make

In [7]:
import pandas as pd
from datetime import datetime as dt, timedelta
AQ_URL = 'https://aqicn.org/data-platform/covid19/report/10248-8ad6289d/2020/waqi-covid19-airqualitydata-2020.csv'
update_date = dt.now() - timedelta(10)

dtypes={'Country': str, 'City': str, 'Specie': str, 'count': int, 'min': float, 'max': float, 'median': float, 'variance': float}

In [12]:
# df = pd.read_csv(AQ_URL, delimiter=',', dtype=dtypes, engine='python', comment='#', error_bad_lines=False,low_memory=False)

In [10]:
chunksize = 50000
reader = pd.read_csv(
    AQ_URL, delimiter=',', engine='python', 
    comment='#', error_bad_lines=False, 
    chunksize=chunksize, iterator=True
)

dfs = []
for df_chunk in reader:
    df_chunk = df_chunk[df_chunk['City'].isin(City.objects.values_list('name', flat=True))]
    df_chunk['date_as_ts'] = pd.to_datetime(df_chunk['Date'])
    df_chunk = df_chunk[df_chunk['date_as_ts'] >= update_date]
    dfs.append(df_chunk)

df = pd.concat(dfs)

In [55]:
import pandas as pd 
import pickle

in_path = "" #Path where the large file is
out_path = config('ROOTPATH') + 'casestudy/update/data/aq/' #Path to save the pickle files to
chunk_size = 400000 #size of chunks relies on your available memory
separator = "~"

reader = pd.read_csv(AQ_URL, sep=separator, chunksize=chunk_size, low_memory=False)

for i, chunk in enumerate(reader):
    out_file = out_path + "/data_{}.pkl".format(i+1)
    with open(out_file, "wb") as f:
        pickle.dump(chunk,f,pickle.HIGHEST_PROTOCOL)

In [56]:
import glob
pickle_path = "" #Same Path as out_path i.e. where the pickle files are

data_p_files=[]
for name in glob.glob(out_path + "/data_*.pkl"):
   data_p_files.append(name)


dfs = []
for i in range(len(data_p_files)):
    df_chunk = pd.read_pickle(data_p_files[i], delimiter=',', engine='python', comment='#', error_bad_lines=False)
    print (df_chunk.columns)
    df_chunk = df_chunk[df_chunk['City'].isin(City.objects.values_list('name', flat=True))]
    df_chunk['date_as_ts'] = pd.to_datetime(df_chunk['Date'])
    df_chunk = df_chunk[df_chunk['date_as_ts'] >= update_date]
    dfs.append(df_chunk)

df_chunked = pd.concat(dfs)

TypeError: read_pickle() got an unexpected keyword argument 'delimiter'

In [ ]:
chunksize = 20000
for chunk in pd.read_csv('./movielens/ratings.csv', chunksize=chunksize, iterator=True):
    do_something_with_chunk(chunk)

In [42]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968580 entries, 0 to 968579
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Date      968580 non-null  object 
 1   Country   968580 non-null  object 
 2   City      968580 non-null  object 
 3   Specie    968580 non-null  object 
 4   count     968580 non-null  int64  
 5   min       968580 non-null  float64
 6   max       968580 non-null  float64
 7   median    968580 non-null  float64
 8   variance  968580 non-null  float64
dtypes: float64(4), int64(1), object(4)
memory usage: 274.1 MB


In [9]:
auto_update(test=True)

this is live!
logger
udpating italy
Running italy...
udpating braz
Running braz...


KeyboardInterrupt: 

In [6]:
from casestudy.models import Measurements

In [7]:
Measurements.objects.latest('date').date

datetime.datetime(2020, 6, 9, 12, 0)

In [8]:
baseframe

,region_id,country_id,region_code,region_name,country_code,country,date,cases,deaths,tests,...,genito,childbirth,perinatal,congenital,other,external,visitors,travel_year,gdp,gdp_year
10092,282,110,ABR,Abruzzo,ITA,Italy,2020-01-01,NaN,NaN,NaN,...,442.0,1.0,16.0,19.0,384.0,2059,181458.0,2017.0,4.560860e+10,2016.0
10093,282,110,ABR,Abruzzo,ITA,Italy,2020-01-02,NaN,NaN,NaN,...,442.0,1.0,16.0,19.0,384.0,2059,181458.0,2017.0,4.560860e+10,2016.0
10094,282,110,ABR,Abruzzo,ITA,Italy,2020-01-03,NaN,NaN,NaN,...,442.0,1.0,16.0,19.0,384.0,2059,181458.0,2017.0,4.560860e+10,2016.0
10095,282,110,ABR,Abruzzo,ITA,Italy,2020-01-04,NaN,NaN,NaN,...,442.0,1.0,16.0,19.0,384.0,2059,181458.0,2017.0,4.560860e+10,2016.0
10096,282,110,ABR,Abruzzo,ITA,Italy,2020-01-05,NaN,NaN,NaN,...,442.0,1.0,16.0,19.0,384.0,2059,181458.0,2017.0,4.560860e+10,2016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45406,519,248,ZWE,Zimbabwe,ZWE,Zimbabwe,2020-06-09,314.0,4.0,22253.0,...,0.0,1180.0,8411.0,0.0,41.0,405,2423000.0,2017.0,3.770507e+10,2016.0
45407,519,248,ZWE,Zimbabwe,ZWE,Zimbabwe,2020-06-10,320.0,4.0,22805.0,...,0.0,1180.0,8411.0,0.0,41.0,405,2423000.0,2017.0,3.770507e+10,2016.0
45408,519,248,ZWE,Zimbabwe,ZWE,Zimbabwe,2020-06-11,332.0,4.0,NaN,...,0.0,1180.0,8411.0,0.0,41.0,405,2423000.0,2017.0,3.770507e+10,2016.0
45409,519,248,ZWE,Zimbabwe,ZWE,Zimbabwe,2020-06-12,343.0,4.0,NaN,...,0.0,1180.0,8411.0,0.0,41.0,405,2423000.0,2017.0,3.770507e+10,2016.0
